In [2]:
# standard modules
import io
import json
import os
from pathlib import Path
import time

# specialized modules
import ee
import geemap
import geopandas as gpd
from pathlib import Path
from tqdm import tqdm

# initialize the Earth Engine module.
ee.Initialize(project='trinity-438000')


# read AOI
dot_slash = Path().cwd()
aoi_path = dot_slash.parent / 'untracked_qgis' / 'aoi.geojson'




In [3]:
aoi = gpd.read_file(aoi_path).to_crs(4326)
gee_json = json.loads(aoi[['geometry']].to_json())
gee_aoi = geemap.geojson_to_ee(gee_json)

# inspect the GeoJSON as an EEObject through geemap.
test_map = geemap.Map()
test_map.centerObject(gee_aoi, 9) # Centering on the AOI instead of an undefined 'region'
test_map.addLayer(gee_aoi, {}, 'AOI')

test_map


Map(center=[41.15499963539352, -122.81974142942383], controls=(WidgetControl(options=['position', 'transparent…

In [4]:
# get extent
minx, miny, maxx, maxy = aoi.total_bounds

verts = [[
[minx, miny],
[minx, maxy],
[maxx, maxy],
[maxx, miny],
[minx, miny]
]]

# make normal float from np.float63
verts = [[float(x), float(y)] for x, y in verts[0]]

verts

[[-122.86506354915134, 41.11218034751449],
 [-122.86506354915134, 41.19781334143016],
 [-122.77436041752618, 41.19781334143016],
 [-122.77436041752618, 41.11218034751449],
 [-122.86506354915134, 41.11218034751449]]

In [5]:
# date range
START = ee.Date('2015-01-01')
END = ee.Date('2025-08-01')

# date and geographic filter
col_filter = ee.Filter.And(
    ee.Filter.geometry(ee.Geometry.Polygon(verts)),
    ee.Filter.date(START, END),
)

# dynamic earth
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(col_filter)


In [14]:
# Create a visualization that blends DW class label with probability.
# Define list pairs of DW LULC label and color.
class_names = [
    'water',
    'trees',
    'grass',
    'flooded_vegetation',
    'crops',
    'shrub_and_scrub',
    'built',
    'bare',
    'snow_and_ice',
]

class_values = list(range(len(class_names)))



In [ ]:
tasks = []
for year in tqdm(range(2013, 2026)):
    # filter for June and AOI
    start = ee.Date(f'{year}-06-01')
    end = start.advance(1, 'month')
    lil_filter = ee.Filter.And(
    ee.Filter.geometry(ee.Geometry.Polygon(verts)),
    ee.Filter.date(start, end),
    )

    # s2 median composite
    dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(lil_filter)
    
    # if there are no images skip
    if dw_col.size().eq(0):
      print(f'{year}: no images, skipping')
      continue

    # Create an RGB image of the label (most likely class) on [0, 1].
    dw_rgb = (
      dw_col.select('label').median()
      .toByte().rename('dw_label')
      .set({
          'class_values': class_values,
          'class_names': class_names
      })
    )

    # download
    export_params = {
      'image': dw_rgb.clip(gee_aoi),
      'description': f'dw_{year}',
      'folder': 'nr218',
      'fileNamePrefix': f'dw_{year}',
      'scale': 10,
      'region': gee_aoi.geometry(),
      'fileFormat': 'GeoTIFF',
      'maxPixels': 1e12 
    }
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()
    tasks.append(task)

100%|██████████| 5/5 [00:00<00:00, 1633.42it/s]

2013: no images, skipping
2014: no images, skipping
2015: no images, skipping
2016: no images, skipping
2017: no images, skipping


In [22]:
for task in tasks:
    info = task.status()
    print(f"{info['description']}: {info['state']}", info.get('progress', ''))


dw_2016: FAILED 
dw_2017: FAILED 
dw_2018: COMPLETED 
dw_2019: COMPLETED 
dw_2020: COMPLETED 
dw_2021: RUNNING 
dw_2022: RUNNING 
dw_2023: READY 
dw_2024: READY 
dw_2025: READY 
